In [1]:
import pandas as pd
import os
diretorio_destino = '.'
df = pd.read_excel('../ContratosPublicos2024.xlsx')
df.head()
dataframes_por_coluna = {}
for nome_coluna in df.columns:
    novo_df = df[[nome_coluna]].copy()
    nome_arquivo_limpo = nome_coluna.replace(' ', '_').replace('/', '_')
    caminho_completo = os.path.join(diretorio_destino, f'{nome_arquivo_limpo}.csv')
    novo_df.to_csv(caminho_completo, index=False)

In [2]:
import pandas as pd
import os

# --- Configurações ---
ARQUIVO_ADJUDICANTES = 'adjudicante.csv'
ARQUIVO_ADJUDICATARIOS = 'adjudicatarios.csv'
ARQUIVO_SAIDA = 'entidades.csv'

# Nomes das colunas combinadas em CADA arquivo (Ajuste se estes nomes não estiverem corretos!)
COLUNA_COMBINADA_ADJUDICANTE = 'adjudicante'
COLUNA_COMBINADA_ADJUDICATARIO = 'adjudicatarios'

COLUNAS_PARA_UNICIDADE = ['NIF', 'Nome'] 
NOME_COLUNA_UNIFICADA = 'Entidade_Completa' # Nome temporário para a coluna combinada após o carregamento

# --- 1. Carregar, Selecionar e Renomear as Colunas ---
try:
    # 1.1 Carregar APENAS a coluna relevante de cada arquivo
    df_adjudicantes = pd.read_csv(ARQUIVO_ADJUDICANTES, 
                                  usecols=[COLUNA_COMBINADA_ADJUDICANTE])
    
    df_adjudicatarios = pd.read_csv(ARQUIVO_ADJUDICATARIOS, 
                                    usecols=[COLUNA_COMBINADA_ADJUDICATARIO])
    
    # 1.2 Renomear a coluna em cada DataFrame para um nome UNIFICADO
    df_adjudicantes.rename(columns={COLUNA_COMBINADA_ADJUDICANTE: NOME_COLUNA_UNIFICADA}, 
                           inplace=True)
    df_adjudicatarios.rename(columns={COLUNA_COMBINADA_ADJUDICATARIO: NOME_COLUNA_UNIFICADA}, 
                             inplace=True)
    
    print(f"Lido '{ARQUIVO_ADJUDICANTES}' ({len(df_adjudicantes)} linhas) e coluna renomeada para '{NOME_COLUNA_UNIFICADA}'.")
    print(f"Lido '{ARQUIVO_ADJUDICATARIOS}' ({len(df_adjudicatarios)} linhas) e coluna renomeada para '{NOME_COLUNA_UNIFICADA}'.")

except FileNotFoundError as e:
    print(f"ERRO: Não foi possível encontrar o arquivo: {e.filename}. Certifique-se de que os arquivos estão no mesmo diretório do script.")
    exit()
except ValueError:
    print(f"ERRO: Verifique os nomes das colunas. As colunas de entrada ('{COLUNA_COMBINADA_ADJUDICANTE}' e '{COLUNA_COMBINADA_ADJUDICATARIO}') não foram encontradas.")
    exit()

# --- 2. Divisão da Coluna Unificada ---

# Processar ambos os DataFrames (agora ambos têm a coluna NOME_COLUNA_UNIFICADA)
for df in [df_adjudicantes, df_adjudicatarios]:
    
    # Padronizar a coluna combinada (remover espaços extras e preencher vazios)
    df[NOME_COLUNA_UNIFICADA] = df[NOME_COLUNA_UNIFICADA].fillna('').astype(str).str.strip()
    
    # Usar .str.split() para dividir a coluna pelo primeiro ' - ' encontrado
    # n=1 garante que divide apenas na primeira ocorrência: "505111667" e "Urbe - Consultores..."
    df[['NIF', 'Nome']] = df[NOME_COLUNA_UNIFICADA].str.split(' - ', n=1, expand=True)

    # Limpar espaços em branco adicionais que possam surgir após a divisão
    df['NIF'] = df['NIF'].str.strip()
    df['Nome'] = df['Nome'].str.strip()
    
    # Remover a coluna combinada original
    df.drop(columns=[NOME_COLUNA_UNIFICADA], inplace=True)


# --- 3. Concatenação e Limpeza ---

# Concatenar (empilhar) os DataFrames
df_entidades_com_duplicatas = pd.concat(
    [df_adjudicantes, df_adjudicatarios], 
    ignore_index=True
)

linhas_totais = len(df_entidades_com_duplicatas)
print(f"\nTotal de linhas combinadas (com duplicatas): {linhas_totais}")

# Tratar casos onde NIF/Nome pode ser NaN após a divisão
df_entidades_com_duplicatas['NIF'] = df_entidades_com_duplicatas['NIF'].fillna('')
df_entidades_com_duplicatas['Nome'] = df_entidades_com_duplicatas['Nome'].fillna('')


# --- 4. Garantir Unicidade (Remover Duplicatas) ---

# Remover duplicatas com base no par (NIF, Nome)
df_entidades_unicas = df_entidades_com_duplicatas.drop_duplicates(
    subset=COLUNAS_PARA_UNICIDADE, 
    keep='first'
)

linhas_unicas_sem_id = len(df_entidades_unicas)
duplicatas_removidas = linhas_totais - linhas_unicas_sem_id

print(f"Duplicatas removidas (baseado em NIF e Nome): {duplicatas_removidas}")

# --- 5. Gerar Coluna ID Único e Reordenar ---

df_entidades_unicas = df_entidades_unicas.reset_index(drop=True)
df_entidades_unicas['id'] = df_entidades_unicas.index + 1

# Selecionar e reordenar as colunas finais (id, NIF, Nome)
df_entidades_final = df_entidades_unicas[['id', 'NIF', 'Nome']]

# --- 6. Guardar o CSV de Saída ---
df_entidades_final.to_csv(ARQUIVO_SAIDA, index=False)

# --- Fim ---
print(f"\n✨ SUCESSO! O arquivo final '{ARQUIVO_SAIDA}' foi criado com {len(df_entidades_final)} linhas únicas.")
    # Us

Lido 'adjudicante.csv' (21748 linhas) e coluna renomeada para 'Entidade_Completa'.
Lido 'adjudicatarios.csv' (21748 linhas) e coluna renomeada para 'Entidade_Completa'.

Total de linhas combinadas (com duplicatas): 43496
Duplicatas removidas (baseado em NIF e Nome): 32091

✨ SUCESSO! O arquivo final 'entidades.csv' foi criado com 11405 linhas únicas.


In [3]:
import pandas as pd
import os

# Lista dos caminhos completos dos ficheiros a processar
FICHEIROS_A_PROCESSAR = [
    'tipoprocedimento.csv',
    '/home/afonso/bioinf/Vscode/sql/projeto/dozero/tabelas2/fundamentacao.csv',
    '/home/afonso/bioinf/Vscode/sql/projeto/dozero/tabelas2/DescrAcordoQuadro.csv',
    '/home/afonso/bioinf/Vscode/sql/projeto/dozero/tabelas2/localExecucao.csv',
    '/home/afonso/bioinf/Vscode/sql/projeto/dozero/tabelas2/tipoContrato.csv'
]

# Itera sobre cada ficheiro na lista
for caminho_completo in FICHEIROS_A_PROCESSAR:
    
    # Extrai apenas o nome do ficheiro para mensagens
    nome_ficheiro = os.path.basename(caminho_completo)
    
    print(f"\n--- Processando: {nome_ficheiro} ---")

    try:
        # 1. Carregar o DataFrame
        df = pd.read_csv(caminho_completo)
        linhas_originais = len(df)
        print(f"Linhas originais: {linhas_originais}")
        
    except FileNotFoundError:
        print(f"⚠️ ERRO: Ficheiro não encontrado no caminho: {caminho_completo}")
        continue # Passa para o próximo ficheiro na lista
    
    # 2. Remover Linhas com Nulos (NULLs/NaNs)
    
    # Antes da remoção
    nulos_antes = df.isnull().any(axis=1).sum()
    
    # Remove qualquer linha onde pelo menos um valor seja nulo (NaN)
    df.dropna(how='any', inplace=True)
    
    nulos_removidos = nulos_antes
    
    print(f"Linhas com nulos removidas: {nulos_removidos}")
    
    # 3. Remover Repetidos (Duplicatas)
    
    # Antes da remoção
    duplicatas_antes = df.duplicated().sum()
    
    # Remove linhas duplicadas (mantém a primeira ocorrência)
    df.drop_duplicates(keep='first', inplace=True)
    
    duplicatas_removidas = duplicatas_antes
    
    print(f"Linhas duplicadas removidas: {duplicatas_removidas}")
    
    # 4. Criar a Coluna ID Único
    
    # Resetar o índice para garantir uma sequência limpa (de 0 a N-1)
    df = df.reset_index(drop=True)
    
    # Criar a coluna 'ID' começando em 1
    df['ID'] = df.index + 1
    
    # 5. Reordenar as colunas para colocar o ID no início
    
    colunas_novas = ['ID'] + [col for col in df.columns if col != 'ID']
    df = df[colunas_novas]
    
    # 6. Salvar o DataFrame processado (sobrescrevendo o original, ou use um novo nome)
    df.to_csv(caminho_completo, index=False)
    
    print(f"Linhas finais: {len(df)}")
    print(f"✅ Ficheiro '{nome_ficheiro}' processado e salvo com a coluna 'ID'.")

print("\n--- Processamento de todos os ficheiros concluído. ---")


--- Processando: tipoprocedimento.csv ---
Linhas originais: 21748
Linhas com nulos removidas: 0
Linhas duplicadas removidas: 21737


Linhas finais: 11
✅ Ficheiro 'tipoprocedimento.csv' processado e salvo com a coluna 'ID'.

--- Processando: fundamentacao.csv ---
Linhas originais: 21748
Linhas com nulos removidas: 137
Linhas duplicadas removidas: 21520
Linhas finais: 91
✅ Ficheiro 'fundamentacao.csv' processado e salvo com a coluna 'ID'.

--- Processando: DescrAcordoQuadro.csv ---
Linhas originais: 21748
Linhas com nulos removidas: 18203
Linhas duplicadas removidas: 3289
Linhas finais: 256
✅ Ficheiro 'DescrAcordoQuadro.csv' processado e salvo com a coluna 'ID'.

--- Processando: localExecucao.csv ---
Linhas originais: 21748
Linhas com nulos removidas: 0
Linhas duplicadas removidas: 21138
Linhas finais: 610
✅ Ficheiro 'localExecucao.csv' processado e salvo com a coluna 'ID'.

--- Processando: tipoContrato.csv ---
Linhas originais: 21748
Linhas com nulos removidas: 0
Linhas duplicadas removidas: 21729
Linhas finais: 19
✅ Ficheiro 'tipoContrato.csv' processado e salvo com a coluna 'ID'.

--- Processamento de todos os fi

In [4]:
import pandas as pd
import os

# --- Configurações ---
CAMINHO_FICHEIRO = '/home/afonso/bioinf/Vscode/sql/projeto/dozero/tabelas2/localExecucao.csv'
COLUNA_COMBINADA = 'localExecucao'
NOVAS_COLUNAS = ['pais', 'distrito', 'concelho']

print(f"--- Processando: {os.path.basename(CAMINHO_FICHEIRO)} ---")

try:
    # 1. Carregar o DataFrame
    df = pd.read_csv(CAMINHO_FICHEIRO)
    linhas_originais = len(df)
    print(f"Linhas originais: {linhas_originais}")
    
    if COLUNA_COMBINADA not in df.columns:
        print(f"⚠️ ERRO: A coluna '{COLUNA_COMBINADA}' não foi encontrada no ficheiro. Verifique a escrita.")
        exit()
        
except FileNotFoundError:
    print(f"⚠️ ERRO: Ficheiro não encontrado no caminho: {CAMINHO_FICHEIRO}")
    exit()

# 2. Padronizar a coluna combinada
# Tratar como string e remover espaços antes de dividir
df[COLUNA_COMBINADA] = df[COLUNA_COMBINADA].astype(str).str.strip()

# 3. Dividir a coluna 'localExecucao' (O FIX ESTÁ AQUI!)
# Usamos n=2 para garantir que a divisão ocorra no máximo 2 vezes,
# resultando sempre em 3 colunas (país, distrito, concelho), mesmo que haja mais vírgulas no dado.
df[NOVAS_COLUNAS] = df[COLUNA_COMBINADA].str.split(',', n=2, expand=True)

# 4. Limpeza Pós-Divisão e Remoção da Coluna Original

# Aplicar .str.strip() para remover espaços em branco nas novas colunas
for col in NOVAS_COLUNAS:
    # Tratar valores que possam ter ficado vazios ou NaN
    df[col] = df[col].astype(str).str.strip().replace('nan', '') 
    
# Remover a Coluna Combinada Original
df.drop(columns=[COLUNA_COMBINADA], inplace=True)

# 5. Salvar o DataFrame Atualizado
df.to_csv(CAMINHO_FICHEIRO, index=False)

print(f"\nAs colunas {NOVAS_COLUNAS} foram criadas.")
print(f"Coluna '{COLUNA_COMBINADA}' original foi removida.")
print(f"Linhas finais: {len(df)}")
print(f"✅ Ficheiro '{os.path.basename(CAMINHO_FICHEIRO)}' salvo com as novas colunas de localização.")

--- Processando: localExecucao.csv ---
Linhas originais: 610

As colunas ['pais', 'distrito', 'concelho'] foram criadas.
Coluna 'localExecucao' original foi removida.
Linhas finais: 610
✅ Ficheiro 'localExecucao.csv' salvo com as novas colunas de localização.


In [5]:
import pandas as pd
import os
import re

# --- Configurações ---
CAMINHO_FICHEIRO = '/home/afonso/bioinf/Vscode/sql/projeto/dozero/tabelas2/fundamentacao.csv'
COLUNA_COMBINADA = 'fundamentacao'

print(f"--- Processando: {os.path.basename(CAMINHO_FICHEIRO)} ---")

try:
    # 1. Carregar o DataFrame
    df = pd.read_csv(CAMINHO_FICHEIRO)
    linhas_originais = len(df)
    print(f"Linhas originais: {linhas_originais}")
    
    if COLUNA_COMBINADA not in df.columns:
        print(f"⚠️ ERRO: A coluna '{COLUNA_COMBINADA}' não foi encontrada no ficheiro. Verifique a escrita.")
        exit()
        
except FileNotFoundError:
    print(f"⚠️ ERRO: Ficheiro não encontrado no caminho: {CAMINHO_FICHEIRO}")
    exit()

# 2. Padronizar a coluna combinada e extrair dados
df[COLUNA_COMBINADA] = df[COLUNA_COMBINADA].astype(str).str.strip()

# Expressões regulares para extrair os componentes:
# Captura o número do Artigo, independentemente do que vier antes ou depois.
REGEX_ARTIGO = r'Artigo\s*(\d+\.º|\d+)'

# Captura o número do parágrafo, se existir (n.º X)
REGEX_NUMERO = r'n\.º\s*(\d+)'

# Captura a alínea, se existir (alínea X)
REGEX_ALINEA = r'alínea\s*([a-z])'


# 3. Criar as Novas Colunas usando .str.extract()
# Extrai o Artigo:
df['Artigo'] = df[COLUNA_COMBINADA].str.extract(REGEX_ARTIGO, flags=re.IGNORECASE)

# Extrai o Número (n.º):
df['Numero_n'] = df[COLUNA_COMBINADA].str.extract(REGEX_NUMERO, flags=re.IGNORECASE)

# Extrai a Alínea:
df['Alinea'] = df[COLUNA_COMBINADA].str.extract(REGEX_ALINEA, flags=re.IGNORECASE)


# 4. Limpeza Pós-Extração
# Limpar 'Artigo' para remover o '.º' e garantir que todos sejam apenas números ou string vazia.
df['Artigo'] = df['Artigo'].str.replace('\.º', '', regex=True).fillna('').str.strip()
df['Numero_n'] = df['Numero_n'].fillna('').str.strip()
df['Alinea'] = df['Alinea'].fillna('').str.strip()

# 5. Remover a Coluna Combinada Original
df.drop(columns=[COLUNA_COMBINADA], inplace=True)

# 6. Salvar o DataFrame Atualizado
df.to_csv(CAMINHO_FICHEIRO, index=False)

print(f"\nAs colunas 'Artigo', 'Numero_n' e 'Alinea' foram criadas.")
print(f"Coluna '{COLUNA_COMBINADA}' original foi removida.")
print(f"Linhas finais: {len(df)}")
print(f"✅ Ficheiro '{os.path.basename(CAMINHO_FICHEIRO)}' salvo com as novas colunas de fundamentação legal.")

--- Processando: fundamentacao.csv ---
Linhas originais: 91

As colunas 'Artigo', 'Numero_n' e 'Alinea' foram criadas.
Coluna 'fundamentacao' original foi removida.
Linhas finais: 91
✅ Ficheiro 'fundamentacao.csv' salvo com as novas colunas de fundamentação legal.


<>:52: SyntaxWarning: invalid escape sequence '\.'
<>:52: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipykernel_27266/1607551723.py:52: SyntaxWarning: invalid escape sequence '\.'
  df['Artigo'] = df['Artigo'].str.replace('\.º', '', regex=True).fillna('').str.strip()


In [6]:
import pandas as pd
import os

# --- Configurações ---
# Substitua 'cpv.csv' pelo caminho completo se não estiver no diretório de trabalho
CAMINHO_FICHEIRO = 'cpv.csv' 
COLUNA_COMBINADA = 'cpv'
NOVAS_COLUNAS = ['Codigo_CPV', 'Descricao']
DELIMITADOR = ' - ' # Hífen com espaços ao lado

print(f"--- Processando: {os.path.basename(CAMINHO_FICHEIRO)} ---")

try:
    # 1. Carregar o DataFrame
    df = pd.read_csv(CAMINHO_FICHEIRO)
    linhas_originais = len(df)
    print(f"Linhas originais: {linhas_originais}")
    
    if COLUNA_COMBINADA not in df.columns:
        print(f"⚠️ ERRO: A coluna '{COLUNA_COMBINADA}' não foi encontrada no ficheiro. Verifique a escrita.")
        exit()
        
except FileNotFoundError:
    print(f"⚠️ ERRO: Ficheiro não encontrado no caminho: {CAMINHO_FICHEIRO}")
    exit()

# 2. Padronizar a coluna combinada
# Tratar como string e remover espaços antes de dividir
df[COLUNA_COMBINADA] = df[COLUNA_COMBINADA].astype(str).str.strip()

# 3. Dividir a coluna 'cpv'
# Usamos n=1 para garantir que a divisão ocorra apenas na primeira ocorrência de ' - ',
# resultando em 2 colunas: Código e Descrição.
df[NOVAS_COLUNAS] = df[COLUNA_COMBINADA].str.split(DELIMITADOR, n=1, expand=True)

# 4. Remover Duplicatas (Com base no código e na descrição)
# Os seus dados têm exemplos de duplicatas que devem ser removidas (90611000-3).
df_antes_duplicatas = len(df)
df.drop_duplicates(subset=NOVAS_COLUNAS, keep='first', inplace=True)
duplicatas_removidas = df_antes_duplicatas - len(df)
print(f"Linhas duplicadas removidas: {duplicatas_removidas}")


# 5. Limpeza Pós-Divisão
for col in NOVAS_COLUNAS:
    # Aplicar .str.strip() para remover espaços e tratar NaNs como string vazia
    df[col] = df[col].astype(str).str.strip().replace('nan', '') 
    
# 6. Adicionar a coluna ID (se ela não existir, assumindo que foi o passo anterior)
if 'ID' not in df.columns:
    df = df.reset_index(drop=True)
    df['ID'] = df.index + 1
    colunas_finais = ['ID'] + NOVAS_COLUNAS
    df = df[colunas_finais]


# 7. Remover a Coluna Combinada Original
#df.drop(columns=[COLUNA_COMBINADA], inplace=True)

# 8. Salvar o DataFrame Atualizado
df.to_csv(CAMINHO_FICHEIRO, index=False)

print(f"\nAs colunas {NOVAS_COLUNAS} foram criadas.")
print(f"Coluna '{COLUNA_COMBINADA}' original foi removida.")
print(f"Linhas finais: {len(df)}")
print(f"✅ Ficheiro '{os.path.basename(CAMINHO_FICHEIRO)}' salvo com as novas colunas de CPV.")

--- Processando: cpv.csv ---
Linhas originais: 21748
Linhas duplicadas removidas: 19480

As colunas ['Codigo_CPV', 'Descricao'] foram criadas.
Coluna 'cpv' original foi removida.
Linhas finais: 2268
✅ Ficheiro 'cpv.csv' salvo com as novas colunas de CPV.


In [7]:
import pandas as pd

def manter_primeira_ocorrencia_na_segunda_coluna(nome_arquivo_entrada, nome_arquivo_saida):
    """
    Remove linhas duplicadas focando EXCLUSIVAMENTE na segunda coluna (índice 1).
    Mantém a primeira ocorrência do valor na 2ª coluna e descarta as subsequentes, 
    independentemente dos valores nas outras colunas.
    """
    try:
        # 1. Carregar o arquivo CSV
        df = pd.read_csv(nome_arquivo_entrada)

        # 2. Identificar a segunda coluna (índice 1)
        if len(df.columns) < 2:
            print("❌ ERRO: O arquivo não tem pelo menos duas colunas.")
            return

        nome_segunda_coluna = df.columns[1]

        # 3. Remover duplicatas APENAS na 2ª coluna
        # O parâmetro 'subset' instrui o pandas a ignorar todas as outras colunas.
        # 'keep="first"' garante a preservação da primeira aparição.
        df_limpo = df.drop_duplicates(
            subset=[nome_segunda_coluna], 
            keep='first'
        )
        
        # 4. Reajustar o índice (opcional)
        df_limpo = df_limpo.reset_index(drop=True)

        # 5. Salvar o DataFrame limpo em um novo arquivo CSV
        df_limpo.to_csv(nome_arquivo_saida, index=False)

        num_linhas_originais = len(df)
        num_linhas_finais = len(df_limpo)
        
        print(f"✅ Processamento concluído: Remoção focada na coluna '{nome_segunda_coluna}'.")
        print(f"   Arquivo de entrada (linhas): {num_linhas_originais}")
        print(f"   Arquivo de saída (linhas): {num_linhas_finais}")
        print(f"   ➡️ Total de linhas removidas: {num_linhas_originais - num_linhas_finais}")
            
    except FileNotFoundError:
        print(f"❌ ERRO: O arquivo '{nome_arquivo_entrada}' não foi encontrado.")
    except Exception as e:
        print(f"❌ Ocorreu um erro: {e}")

# --- Chamada da função ---
manter_primeira_ocorrencia_na_segunda_coluna('cpv.csv', 'cpv_unico_pela_coluna_2.csv')

✅ Processamento concluído: Remoção focada na coluna 'Codigo_CPV'.
   Arquivo de entrada (linhas): 2268
   Arquivo de saída (linhas): 2127
   ➡️ Total de linhas removidas: 141


In [8]:
import pandas as pd

def mostrar_valores_repetidos_segunda_coluna(nome_arquivo):
    """
    Carrega o CSV e exibe a contagem de todos os valores repetidos na segunda coluna.
    """
    try:
        # 1. Carregar o arquivo CSV
        df = pd.read_csv(nome_arquivo)

        # 2. Identificar e selecionar a segunda coluna (índice 1)
        if len(df.columns) < 2:
            print("❌ ERRO: O arquivo tem menos de duas colunas.")
            return

        nome_segunda_coluna = df.columns[2]
        coluna = df[nome_segunda_coluna]
        
        print(f"--- Análise da Coluna: '{nome_segunda_coluna}' (Índice 1) ---")

        # 3. Identificar todas as ocorrências de valores duplicados
        # duplicated(keep=False) marca TODAS as linhas onde o valor se repete.
        duplicatas_series = coluna[coluna.duplicated(keep=False)]

        if duplicatas_series.empty:
            print("\n✅ Não foram encontrados valores repetidos na segunda coluna.")
        else:
            # 4. Contar a frequência de cada valor repetido
            # O value_counts() lista os valores duplicados e a sua contagem.
            frequencia_duplicatas = duplicatas_series.value_counts()
            
            print(f"\n❌ {len(frequencia_duplicatas)} VALORES DIFERENTES ENCONTRADOS REPETIDOS:")
            print("Abaixo está a lista desses valores e a frequência (quantas vezes aparecem no total):")
            
            # Exibir os resultados de forma clara
            print("\nValores Repetidos (2ª Coluna) | Contagem Total")
            print("-----------------------------------|----------------")
            # Loop para imprimir de forma formatada
            for valor, contagem in frequencia_duplicatas.items():
                print(f"{valor:<35}| {contagem}")
            
            print(f"\nTotal de linhas que contêm valores duplicados (todas as ocorrências): {len(duplicatas_series)}")
            print(f"Total de valores únicos na coluna: {coluna.nunique()}")

    except FileNotFoundError:
        print(f"\n❌ ERRO: O arquivo '{nome_arquivo}' não foi encontrado.")
        print("Por favor, garanta que o 'cpv.csv' está no mesmo diretório do seu script.")
    except Exception as e:
        print(f"❌ Ocorreu um erro inesperado: {e}")

# --- Chamada da função ---
mostrar_valores_repetidos_segunda_coluna('cpv_unico_pela_coluna_2.csv')

--- Análise da Coluna: 'Descricao' (Índice 1) ---

✅ Não foram encontrados valores repetidos na segunda coluna.


In [9]:
"""
Criacao contratos

"""
import pandas as pd

nome_arquivo = 'contratos.csv'

df1 = pd.read_csv("idcontrato.csv")
df2 = pd.read_csv("prazoExecucao.csv")
df3 = pd.read_csv("precoContratual.csv")
df4 = pd.read_csv("dataCelebracaoContrato.csv")
df5 = pd.read_csv("dataPublicacao.csv")
df6 = pd.read_csv("ProcedimentoCentralizado.csv")
df7 = pd.read_csv("objectoContrato.csv")

# 1. Corrija o nome da coluna chave em todos os DataFrames (VITAL)
# Se ainda tiver dúvidas sobre o KeyError, execute este bloco:
dataframes_para_concatenar = [df1, df2, df3, df4, df5, df6, df7]

# O argumento 'axis=1' garante que os DataFrames são unidos horizontalmente
contratos = pd.concat(dataframes_para_concatenar, axis=1)
print("\n🎉 DataFrame 'contratos' unido corretamente!")
contratos.to_csv('contratos.csv',index=False,encoding='utf-8')
print(contratos.head())


🎉 DataFrame 'contratos' unido corretamente!
   idcontrato  prazoExecucao  precoContratual dataCelebracaoContrato  \
0    10424261            366          3918.75             2024-01-01   
1    10424271            366           467.95             2024-01-02   
2    10424433             30         27849.00             2024-01-02   
3    10424474            365         11520.00             2024-01-02   
4    10424593            365         19248.00             2024-01-02   

  dataPublicacao ProcedimentoCentralizado  \
0     2024-01-01                      Não   
1     2024-01-02                      Não   
2     2024-01-02                      Não   
3     2024-01-02                      Não   
4     2024-01-02                      Não   

                                     objectoContrato  
0  Seguro de Acidentes de Trabalho para os Funcio...  
1          Seguro Automóvel - FIAT 500L/1.3MJ LOUNGE  
2              Ligação EN 342 a placa de S. Martinho  
3  Serviços de limpeza de berma

In [10]:
import pandas as pd
import numpy as np

nome_arquivo_referencia = "tipoContrato.csv"
nome_arquivo_saida = 'contratos_final_processado.csv'

# -----------------------------------------------------------
## PASSO 2: MAPEAMENTO DO ID ('tipoContrato' por ID numérico)
# -----------------------------------------------------------

print("\n## 2. Mapeamento da coluna 'tipoContrato'")

# 1. Carregar o DataFrame de referência
try:
    df_ref_tipo = pd.read_csv(nome_arquivo_referencia)
    
    # Preparações para a junção:
    
    # Renomear o ID na tabela de referência (evita conflitos e prepara o nome final)
    df_ref_tipo.rename(columns={'ID': 'id_tipoContrato_num'}, inplace=True)
    
    # Assegurar que as colunas de junção têm o mesmo tipo de dado e estão limpas (VITAL)
    contratos['tipoContrato'] = contratos['tipoContrato'].astype(str).str.strip()
    df_ref_tipo['tipoContrato'] = df_ref_tipo['tipoContrato'].astype(str).str.strip()

    # 2. Junção (Merge) para adicionar o ID numérico
    # Juntamos usando o nome por extenso como chave
    contratos = contratos.merge(
        df_ref_tipo[['tipoContrato', 'id_tipoContrato_num']],
        on='tipoContrato',  
        how='left'          
    )

    # 3. Limpeza Final:
    # A) Remover a coluna antiga (nome por extenso)
    contratos.drop(columns=['tipoContrato'], inplace=True)

    # B) Renomear a nova coluna ID para o nome final
    contratos.rename(columns={'id_tipoContrato_num': 'id_tipoContrato'}, inplace=True)

    print("✅ Mapeamento concluído: 'tipoContrato' substituído por 'id_tipoContrato'")

except FileNotFoundError:
    print(f"⚠️ Aviso: Arquivo de referência '{nome_arquivo_referencia}' não encontrado. Mapeamento ignorado.")

# -----------------------------------------------------------
## PASSO 3: EXPORTAÇÃO PARA CSV
# -----------------------------------------------------------

# Exporta o DataFrame 'contratos' (agora com o ID numérico) para o arquivo CSV
contratos.to_csv(nome_arquivo_saida, index=False, encoding='utf-8')

print(f"\n✅ DataFrame final guardado com sucesso em: **{nome_arquivo_saida}**")
print("\nPrimeiras linhas do resultado final:")
print(contratos.head())


## 2. Mapeamento da coluna 'tipoContrato'


KeyError: 'tipoContrato'